In [ ]:
%load_ext autoreload
%autoreload 2

import music21 as mu
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from data_helper import *
from music_helper import *

In [ ]:
pathes = [
    "./tmp/PianoFingeringDataset/FingeringFiles/{0:03d}-1_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/{0:03d}-2_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/{0:03d}-5_fingering.txt",
    "./tmp/PianoFingeringDataset/FingeringFiles/{0:03d}-8_fingering.txt",
    './tmp/Result_FHMM1/{0:03d}-1_fingering.txt',
    './tmp/Result_FHMM2/{0:03d}-1_fingering.txt',
    './tmp/Result_FHMM3/{0:03d}-1_fingering.txt'
]
titles = [
    "PIG_1","PIG_2","PIG_5","PIG_8",
    "FHMM1","FHMM2","FHMM3",
]

In [ ]:
# Bachセット10曲分
tables = {}
for i in range(10):
    tables[i] = [read_PIG(path.format(i + 1)) for path in pathes]
    
df = pd.DataFrame()
for p in range(len(pathes)):
    sum = np.zeros(5)
    for m in range(len(tables)):
        table = tables[m][p]
        sum += table[table.ch == 0].groupby('finger').finger.count()
    df = df.append(sum)

df.index = titles
sns.heatmap(df, annot=True, cmap="viridis")

In [ ]:
costs = np.zeros(shape = (len(pathes), 10))
for p in range(len(pathes)):
    sum = np.zeros(5)
    for m in range(len(tables)):
        table = tables[m][p]
        notes = table[table.ch == 0]
        rest = False
        for pi in range(1, len(notes)):
            n_ = notes.iloc[pi - 1]
            n = notes.iloc[pi]
            pd_ = n.pos - n_.pos
            fi = n.fingers[-1] - 1
            fi_ = n_.fingers[-1] - 1
            fd_ = fi - fi_
            td_ = n.t0 - n_.t1
            if td_ >= 1 / 32: # 休符を挟まない
                rest = True
                continue
            if not rest:
                costs[p, m] += np.sqrt(abs(pd_ - fd_))
            rest = False

In [ ]:
dfa = pd.DataFrame(costs, index = titles, columns = range(1, 11))
sns.heatmap(dfa)

### データ生成

In [ ]:
import os

dirname = "./tmp/PianoFingeringDataset/FingeringFiles"
names = [ path for path in os.listdir(dirname) if ".txt" in path ]
pigs = [ read_PIG(os.path.join(dirname, name)) for name in names ]

In [ ]:
columns = ["寄せ", "拡げ", "潜り", "超え", "その他"]
tables = pigs
actions = np.zeros(shape = (len(tables), len(columns)))

widthX = 15
nCnt = 3*(2*widthX+1)

def getDiff(m_, m):
    o = np.array(key_pos2(m)) - np.array(key_pos2(m_))
    return int(3*(np.clip(o[0], -widthX, widthX) + widthX) + (o[1] + 1))

trSilent = np.zeros(shape = (5,5))
trChange = np.zeros(shape = (5,5))

diffSilent = np.zeros(shape = (25, nCnt))
diffChange = np.zeros(shape = (25, nCnt))

for p, table in enumerate(tables):
    notes = table[table.ch == 0]
    rest = False
    for pi in range(1, len(notes)):
        n_ = notes.iloc[pi - 1]
        n = notes.iloc[pi]
        td_ = n.t0 - n_.t1
        fd_ = n.fingers[-1] - n_.fingers[-1]
        pd_ = n.pos - n_.pos
        if td_ >= 1 / 32: # 休符を挟まない
            rest = True
            continue
        if not rest:
            move = getMoveTypeByPIGNote(n, n_)
            actions[p, move] += 1            
            fi = n.fingers[-1] - 1
            fi_ = n_.fingers[-1] - 1
            if move != 0:
                trSilent[fi_, fi] += 1
                diffSilent[fi_ * 5 + fi, getDiff(n_.midi, n.midi)] += 1
            else:
                trChange[fi_, fi] += 1
                diffChange[fi_ * 5 + fi, getDiff(n_.midi, n.midi)] += 1
        rest = False


In [ ]:
dfa = pd.DataFrame(actions, columns=columns, index = names[0:len(tables)])
dfa

In [ ]:
trProbSilent = np.apply_along_axis(lambda c: c / c.sum(), 1, trSilent)
outProbSilent = np.apply_along_axis(lambda c: c / c.sum(), 1, diffSilent)
trProbChange = np.apply_along_axis(lambda c: c / c.sum(), 1, trChange)
outProbChange = np.apply_along_axis(lambda c: c / c.sum(), 1, diffChange)

In [ ]:
np.savetxt("./data/trProbSilent.txt", trProbSilent)
np.savetxt("./data/outProbSilent.txt", outProbSilent)
np.savetxt("./data/trProbChange.txt", trProbChange)
np.savetxt("./data/outProbChange.txt", outProbChange)

In [ ]:
sns.heatmap(outProbSilent)

In [ ]:
sns.heatmap(outProbChange)